In [1]:
    import pandas as pd

In [2]:
import numpy as np

In [6]:
df=pd.read_csv("teja.csv")

In [7]:
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [22]:
df=df.drop(columns=["Loan_ID","Gender","Married","Dependents","CoapplicantIncome"
                   ])

KeyError: "['Loan_ID', 'Gender', 'Married', 'Dependents'] not found in axis"

In [ ]:
df["Loan_Amount_Term"]=df["Loan_Amount_Term"].fillna(df["Loan_Amount_Term"].mean())
df["LoanAmount"]=df["LoanAmount"].fillna(df["LoanAmount"].mean())
df.head()

In [ ]:
df.isnull().sum()

In [23]:
df

,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,Graduate,No,5849,0.0,146.412162,360.0,1.0,Urban,Y
1,Graduate,No,4583,1508.0,128.000000,360.0,1.0,Rural,N
2,Graduate,Yes,3000,0.0,66.000000,360.0,1.0,Urban,Y
3,Not Graduate,No,2583,2358.0,120.000000,360.0,1.0,Urban,Y
4,Graduate,No,6000,0.0,141.000000,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...
609,Graduate,No,2900,0.0,71.000000,360.0,1.0,Rural,Y
610,Graduate,No,4106,0.0,40.000000,180.0,1.0,Rural,Y
611,Graduate,No,8072,240.0,253.000000,360.0,1.0,Urban,Y
612,Graduate,No,7583,0.0,187.000000,360.0,1.0,Urban,Y


In [30]:

df.shape

(614, 8)

In [31]:
df.dropna(inplace=True)

In [32]:
df.shape

(538, 8)

In [33]:
df.isnull().sum()

Education           0
Self_Employed       0
ApplicantIncome     0
LoanAmount          0
Loan_Amount_Term    0
Credit_History      0
Property_Area       0
Loan_Status         0
dtype: int64

In [34]:
df.describe()

,ApplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
count,538.000000,538.000000,538.000000,538.000000
mean,5432.211896,145.371107,342.323420,0.836431
std,5885.545882,81.119033,64.067407,0.370228
min,150.000000,9.000000,36.000000,0.000000
25%,2896.250000,101.250000,360.000000,1.000000
50%,3858.500000,130.000000,360.000000,1.000000
75%,5820.500000,165.000000,360.000000,1.000000
max,81000.000000,700.000000,480.000000,1.000000


In [134]:
from sklearn.preprocessing import OneHotEncoder

ohe=OneHotEncoder(sparse=False,handle_unknown='ignore')

In [135]:
ohe.fit(df[["Education","Self_Employed","Property_Area"]])
ohe.categories_

[array(['Graduate', 'Not Graduate'], dtype=object),
 array(['No', 'Yes'], dtype=object),
 array(['Rural', 'Semiurban', 'Urban'], dtype=object)]

In [136]:
 from sklearn.compose import make_column_transformer

In [137]:
col=make_column_transformer((OneHotEncoder(categories=ohe.categories_),["Education","Self_Employed","Property_Area"]),remainder="passthrough")

In [138]:
from sklearn.tree import DecisionTreeClassifier

In [139]:
de= DecisionTreeClassifier()

In [140]:
from sklearn.pipeline import make_pipeline

In [141]:
pipe=make_pipeline(col,de)

In [142]:
df.columns

Index(['Education', 'Self_Employed', 'ApplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status'],
      dtype='object')

In [143]:
x=df.drop(columns=["Loan_Status","Loan_Amount_Term"])

In [144]:
y=df.Loan_Status

In [145]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=30)

In [146]:
pipe.fit(x_train,y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories=[array(['Graduate', 'Not Graduate'], dtype=object),
                                                                            array(['No', 'Yes'], dtype=object),
                                                                            array(['Rural', 'Semiurban', 'Urban'], dtype=object)]),
                                                  ['Education', 'Self_Employed',
                                                   'Property_Area'])])),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [147]:
y_pred=pipe.predict(x_test)

In [148]:
from sklearn.metrics import accuracy_score

In [149]:
accuracy_score(y_test,y_pred)

0.7592592592592593

In [150]:
lis=[]
for i in range(1,1000):
    x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=i)
    pipe.fit(x_train,y_train)
    y_pred=pipe.predict(x_test)
    lis.append(accuracy_score(y_test,y_pred))

In [151]:
np.argmax(lis)

941

In [152]:
lis[np.argmax(lis)]

0.8518518518518519

In [153]:
pip=make_pipeline(col,de)
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=941)
pip.fit(x_train,y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories=[array(['Graduate', 'Not Graduate'], dtype=object),
                                                                            array(['No', 'Yes'], dtype=object),
                                                                            array(['Rural', 'Semiurban', 'Urban'], dtype=object)]),
                                                  ['Education', 'Self_Employed',
                                                   'Property_Area'])])),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [154]:
df.columns

Index(['Education', 'Self_Employed', 'ApplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status'],
      dtype='object')

In [155]:
import pickle

In [156]:
pickle.dump(pip,open("teja.pkl","wb"))

In [132]:
x_train

,Education,Self_Employed,ApplicantIncome,LoanAmount,Credit_History,Property_Area
360,Graduate,No,4723,81.0,1.0,Semiurban
87,Graduate,No,2500,104.0,1.0,Semiurban
173,Graduate,No,5708,187.0,1.0,Semiurban
99,Graduate,No,1759,131.0,1.0,Semiurban
454,Graduate,Yes,7085,84.0,1.0,Semiurban
...,...,...,...,...,...,...
591,Graduate,Yes,6000,205.0,1.0,Semiurban
289,Graduate,No,9508,187.0,1.0,Rural
529,Not Graduate,No,6783,130.0,1.0,Semiurban
123,Graduate,No,2957,81.0,1.0,Semiurban


In [133]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 538 entries, 0 to 613
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Education         538 non-null    object 
 1   Self_Employed     538 non-null    object 
 2   ApplicantIncome   538 non-null    int64  
 3   LoanAmount        538 non-null    float64
 4   Loan_Amount_Term  538 non-null    float64
 5   Credit_History    538 non-null    float64
 6   Property_Area     538 non-null    object 
 7   Loan_Status       538 non-null    object 
dtypes: float64(3), int64(1), object(4)
memory usage: 37.8+ KB
